## Objective

Use L2 Lasso regularization, see:
- if $R^2$ improves from 0.56
- whether we avoid crazy `y_pred`s (presumably from overfitting)
- which coefficients drop to 0

In [3]:
from __future__ import division
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20, 5

from helper_functions import dummify_cols_and_baselines, make_alphas, remove_outliers_by_type

In [4]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

## Removing outliers

A standard procedure is to remove values further than 3 standard deviations from the mean. Since I have so many low values and some very high values, I anecdotally think that the low values are very likely to be true, and the high values not so much.

So, I will remove values further than 3 SDs from the median, by type.

Ideally, I would take into account the time dimension. I would like to do so given more time.

In [5]:
df_outliers_removed = remove_outliers_by_type(df_orig, y_col='COMPLETION_HOURS_LOG_10')
df_outliers_removed.shape

helper_functions.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:2454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._where(-key, value, inplace=True)


(508653, 40)

I'm removing ~1.5% of my rows.

## Choosing columns

In [6]:
cols_orig_dataset = ['COMPLETION_HOURS_LOG_10', 'TYPE', 'SubmittedPhoto', 'Property_Type', 'Source', 'neighborhood_from_zip']
cols_census = ['race_white',
     'race_black',
     'race_asian',
     'race_hispanic',
     'race_other',
     'poverty_pop_below_poverty_level',
     'earned_income_per_capita',
     'poverty_pop_w_public_assistance',
     'poverty_pop_w_food_stamps',
     'poverty_pop_w_ssi',
     'school',
     'school_std_dev',
     'housing',
     'housing_std_dev',
     'bedroom',
     'bedroom_std_dev',
     'value',
     'value_std_dev',
     'rent',
     'rent_std_dev',
     'income',
     'income_std_dev']
cols_engineered = ['queue_wk', 'queue_wk_open', 'is_description']

In [7]:
df = df_outliers_removed[cols_orig_dataset + cols_census + cols_engineered]

## Dummify

In [8]:
cols_to_dummify = df.dtypes[df.dtypes == object].index
cols_to_dummify

Index([u'TYPE', u'Property_Type', u'Source', u'neighborhood_from_zip',
       u'school', u'housing'],
      dtype='object')

In [9]:
df_dummified, baseline_cols = dummify_cols_and_baselines(df, cols_to_dummify)

Zoning is baseline 0 6
other is baseline 1 6
Twitter is baseline 2 6
West Roxbury is baseline 3 6
8_6th_grade is baseline 4 6
rent is baseline 5 6


In [8]:
df_dummified.shape

(508653, 253)

## Running model

In [85]:
df.head(2).T.iloc[:16]

,905425,905235
COMPLETION_HOURS_LOG_10,-0.0369997,0.366423
TYPE,Request for Snow Plowing,Misc. Snow Complaint
SubmittedPhoto,True,False
Property_Type,Address,Address
Source,Citizens Connect App,Citizens Connect App
neighborhood_from_zip,Dorchester,East Boston
race_white,0.242399,0.38997
race_black,0.514358,0
race_asian,0.035473,0
race_hispanic,0.0675676,0.562126


In [86]:
df.head(2).T.iloc[16:]

,905425,905235
school,15_hs_diploma,15_hs_diploma
school_std_dev,91.7672,87.8748
housing,own,rent
housing_std_dev,26.8701,153.442
bedroom,3,2
bedroom_std_dev,61.5123,90.1874
value,350000,225000
value_std_dev,20.9794,14.8764
rent,1750,1125
rent_std_dev,19.1622,35.2037


In [10]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score


/home/ec2-user/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df_dummified.drop('COMPLETION_HOURS_LOG_10', axis=1), 
    df_dummified.COMPLETION_HOURS_LOG_10, 
    test_size=0.2, 
    random_state=300
)

In [12]:
pipe = make_pipeline(StandardScaler(), LassoCV())

In [13]:
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [14]:
params = {'lassocv__alphas': make_alphas(-2, -4)}
params = {'lassocv__alphas': [[0.01]]}
model = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=True)
model.fit(X_train, y_train);

Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   26.1s finished


In [15]:
pd.DataFrame(model.cv_results_).T

,0
mean_fit_time,18.4343
mean_score_time,1.16387
mean_test_score,0.544087
mean_train_score,0.546022
param_lassocv__alphas,[0.01]
params,{u'lassocv__alphas': [0.01]}
rank_test_score,1
split0_test_score,0.544087
split0_train_score,0.546022
std_fit_time,0


In [16]:
model.score(X_test, y_test)

0.54159918297330223

In [32]:
y_pred = model.predict(X_test)

In [33]:
mean_squared_error(y_test, y_pred)**0.5

0.75812869114013393

How many coefficients go to 0?

In [17]:
pd.Series(model.best_estimator_.steps[-1][-1].coef_).describe()

count    252.000000
mean      -0.004569
std        0.049632
min       -0.363193
25%        0.000000
50%       -0.000000
75%        0.000000
max        0.145756
dtype: float64

In [29]:
len(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ != 0])

115

In [30]:
len(X_train.columns)

252

About half at `α=0.01`.

Which don't go to zero?

In [27]:
sorted(list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ != 0]))

['Source_Citizens Connect App',
 'Source_Self Service',
 'TYPE_Abandoned Bicycle',
 'TYPE_Abandoned Building',
 'TYPE_Abandoned Vehicles',
 'TYPE_Animal Found',
 'TYPE_Animal Generic Request',
 'TYPE_Animal Lost',
 'TYPE_Bed Bugs',
 'TYPE_Bicycle Issues',
 'TYPE_Breathe Easy',
 'TYPE_Building Inspection Request',
 'TYPE_Call Log',
 'TYPE_Carbon Monoxide',
 'TYPE_Catchbasin',
 'TYPE_Checkin',
 'TYPE_Chronic Dampness/Mold',
 'TYPE_Construction Debris',
 'TYPE_Contractors Complaint',
 'TYPE_Cross Metering - Sub-Metering',
 'TYPE_Egress',
 'TYPE_Electrical',
 'TYPE_Empty Litter Basket',
 'TYPE_Equipment Repair',
 'TYPE_Exceeding Terms of Permit',
 'TYPE_General Comments For An Employee',
 'TYPE_General Comments For a Program or Policy',
 'TYPE_General Lighting Request',
 'TYPE_Graffiti Removal',
 'TYPE_Heat - Excessive  Insufficient',
 'TYPE_Highway Maintenance',
 'TYPE_Housing Discrimination Intake Form',
 'TYPE_Illegal Auto Body Shop',
 'TYPE_Illegal Dumping',
 'TYPE_Illegal Occupancy',


We will want to run a model with just the above features to find out which ones are statistically significant, but we get a sense here that these factors are likely to be signficant:

- when source is from the mobile app or desktop website
- neighborhoods of East Boston and the North End
- the number of issues in the workers' queue at the time

## Conclusion

We didn't get a better $R^2$, which makes sense, since we weren't in an overfit situation anyways when we tried this regularization parameter.

We did find subset our features and got somewhat of an indication which ones are more likely to be significantly correlated to completion time than others. We also avoided crazy predictions that would have affected our $R^2$, at least for this particular random seed.

## Appendix

These columns went to zero at `α=0.01`.

In [31]:
sorted(list(X_train.columns[model.best_estimator_.steps[-1][-1].coef_ == 0]))

['Property_Type_Address',
 'Property_Type_Intersection',
 'Source_Constituent Call',
 'SubmittedPhoto',
 'TYPE_ADA',
 'TYPE_Alert Boston',
 'TYPE_Animal Noise Disturbances',
 'TYPE_Automotive Noise Disturbance',
 'TYPE_BWSC General Request',
 'TYPE_BWSC Pothole',
 'TYPE_Big Buildings Online Request',
 'TYPE_Billing Complaint',
 'TYPE_Bridge Maintenance',
 'TYPE_CE Collection',
 'TYPE_Cemetery Maintenance Request',
 'TYPE_City/State Snow Issues',
 'TYPE_Contractor Complaints',
 'TYPE_Corporate or Community Group Service Day Clean Up',
 'TYPE_Downed Wire',
 'TYPE_Dumpster & Loading Noise Disturbances',
 'TYPE_Fire Department Request',
 'TYPE_Fire Hydrant',
 'TYPE_Fire in Food Establishment',
 'TYPE_Follow-Up',
 'TYPE_Food Alert - Confirmed',
 'TYPE_Food Alert - Unconfirmed',
 'TYPE_General Traffic Engineering Request',
 'TYPE_Ground Maintenance',
 'TYPE_HP Sign Application New',
 'TYPE_HP Sign Application Renewal',
 'TYPE_Heat/Fuel Assistance',
 'TYPE_Idea Collection',
 'TYPE_Knockdown R

What are the coef values?

In [70]:
coef_values = pd.DataFrame({
    'name': X_train.columns,
    'coef': model.best_estimator_.steps[-1][-1].coef_
})

In [75]:
coef_values[coef_values.coef != 0].sort_values('coef')[~coef_values.name.str.contains('TYPE')]

/home/ec2-user/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,coef,name
217,-0.021895,Source_Citizens Connect App
234,-0.000054,neighborhood_from_zip_North End
228,0.006279,neighborhood_from_zip_East Boston
219,0.012568,Source_Self Service
22,0.130668,queue_wk_open
